# Scrping do site Estante Virtual

## Importação de bibliotecas

In [115]:
from selenium import webdriver #Vai me premitir fazer as automações do meu computador para o navegador
from webdriver_manager.chrome import ChromeDriverManager #WebDrive do Chorme que me permitirá acessar todas as infomções da web no Chrome
from selenium.webdriver.chrome.service import Service #Me permitirá ter sempre a versão autualizada do webdriver do Chrome par que eu não precise instalar manualmente
from selenium.webdriver.common.by import By #Me permitirá fazer o scrping do site através das tags, class, xpath...
import pandas as pd #Me permitirará fazer uma base de dados com o dicionario que vai conter as infomações dos livros

In [116]:
service=Service(ChromeDriverManager().install())#Instalando  o webdriver atualizado do Chrome

navegator=webdriver.Chrome(service=service)#Criando conexão do meu notebook para o navegador Chrome

In [136]:
#Listas com as principais informações dos livros
names=[]
quantBooksPage=[] #Saber a quantidade de livros que contém na pagina do site para que eu possa clicar em todos eles e reitrar informações
authors=[]
prices=[]
years=[]
publisher=[]
#Biblioteca que vai armazenar os dados dos livros
books={}

In [128]:
navegator.get(f"https://www.estantevirtual.com.br/busca?editora=claro-enigma&page={1}") #Acessando o site


In [131]:
namesWeb=navegator.find_element(By.CLASS_NAME, "product-list__items").find_elements(By.TAG_NAME, "h2") #Pegando as informações dos nomes de todos os livros 
for nome in namesWeb:
    names.append(nome.text)#Adicionando os nomes na lista de nomes
len(names)#Analisando o tamanho da lista  para que no final eu ver se todas a lista tem a mesma quantidade de elemnetos, desse modo, sendo possível a criação do dataframe.

44

In [132]:
authorsWeb=navegator.find_element(By.CLASS_NAME, "product-list__items").find_elements(By.CLASS_NAME, "product-item__author")#Pegando as informações dos autores de todos os livros 
for author in authorsWeb:
    authors.append(author.text)#Adicionando os autores na lista dos autores
len(authors)

44

In [133]:
pricesWeb=navegator.find_element(By.CLASS_NAME, "product-list__items").find_elements(By.CLASS_NAME, "product-item__text--darken")#Pegando as informações dos nomes de todos os prços
for price in pricesWeb:
    prices.append(price.text)#Adicionando os preços na lista dos preçoes
len(prices)

44

In [134]:
#Analisando as informações de Ano de publicação e editora. Essas informaçoes só sao possíveis de serem coletadas se entra no menu do livro, então fiz o for para entrar no menu
#De todos os livros para extrair essas informações
for c in range(len(names)):
    navegator.find_element(By.CLASS_NAME, "product-list__items").find_elements(By.TAG_NAME, "h2")[c].click() #Entrando no perfi de cada livro com base no nome do livro
    try: #Essa verificação foi feita para se casso o livro não tenha as informações ele coloque no lugar o testo "sem registro"
        ano=navegator.find_element(By.CLASS_NAME, "book-attributes").find_element(By.CLASS_NAME, "book-attributes__item--year").text.split()[1] #Extraindo o Ano de publicação do livros
        years.append(ano) #adicionando o valor do ano na lista
    except:
        years.append("Sem registros")#Caso não tenha o valor do ano, vai ser adicionado o calor "Sem registro"
    try:
        editora=navegator.find_element(By.CLASS_NAME, "book-attributes").find_elements(By.TAG_NAME, "a")[0].text #Extraindo o nome da editora do livro
        publisher.append(editora)#adicionando o valor do ano na lista
    except:
        publisher.append("Sem registros")#Caso não tenha o valor do ano, vai ser adicionado o calor "Sem registro"
    navegator.back()
print(len(years), len(publisher))#Analisando o tamanho das listas para sbaer se todas as listas tem a mesma quantidade elemntos


44 44


In [137]:
#Automatizando para que seja extraido as informações dos livros de todas as paginas do site da Estante Virtual 
names=[]
quantBooksPage=[]
authors=[]
prices=[]
years=[]
publisher=[]
books={}
quantPages=2 #Extrair as informações dos livros das primeiras duas paginas do site 
for c in range(quantPages):
    quantBooksPage=[]
    navegator.get(f"https://www.estantevirtual.com.br/busca?editora=claro-enigma&page={c+1}")
    namesWeb=navegator.find_element(By.CLASS_NAME, "product-list__items").find_elements(By.TAG_NAME, "h2")
    for nome in namesWeb:   
        names.append(nome.text)
        quantBooksPage.append(nome.text)
    authorsWeb=navegator.find_element(By.CLASS_NAME, "product-list__items").find_elements(By.CLASS_NAME, "product-item__author")
    for author in authorsWeb:
        authors.append(author.text)
    pricesWeb=navegator.find_element(By.CLASS_NAME, "product-list__items").find_elements(By.CLASS_NAME, "product-item__text--darken")
    for price in pricesWeb:
        prices.append(price.text)

    for c in range(len(quantBooksPage)):
        navegator.find_element(By.CLASS_NAME, "product-list__items").find_elements(By.TAG_NAME, "h2")[c].click()
        try:
            ano=navegator.find_element(By.CLASS_NAME, "book-attributes").find_element(By.CLASS_NAME, "book-attributes__item--year").text.split()[1]
            editora=navegator.find_element(By.CLASS_NAME, "book-attributes").find_elements(By.TAG_NAME, "a")[0].text
            years.append(ano)
            publisher.append(editora)
        except:
            years.append("Sem registros")
            publisher.append("Sem registros")
        navegator.back()
#Adicionando os falores das listas ao dicionario
books["Nome"]=names
books["Autor"]=authors
books["Editora"]=publisher
books["Ano"]=years
books["preço"]=prices
#Criando o dataframe
df=pd.DataFrame(books)
#criando o arquivo  CSV que será usado na analise
df.to_csv("ScrapingBooks.csv")


    
    

In [138]:
df #Mostrando o dataframe

,Nome,Autor,Editora,Ano,preço
0,A idade média passo a passo,Vincent Carpentier,Claro Enigma,2014,"R$ 10,02"
1,Cidadania: um projeto em construção,Andre Botelho e Lilia Moritz Schwarcz,Claro Enigma,2013,"R$ 6,00"
2,O que é arte contemporânea?,"Klein, Jacky E Klein, Suzy",Claro Enigma,2019,"R$ 15,00"
3,A Grécia antiga passo a passo,Eric Dars,Claro Enigma,2015,"R$ 16,89"
4,A pré-história passo a passo,Hildegard Feist; Colette Swinnen; Loic Mehee,Claro Enigma,2014,"R$ 6,00"
...,...,...,...,...,...
83,A Pré-história Passo a Passo,Collete Swinnen,Claro Enigma,2010,"R$ 25,00"
84,Crescente 1977-1990,Duda Machado,Claro Enigma,1990,"R$ 165,00"
85,Crescente 1977-1990,Duda Machado Com Dedicatoria,Claro Enigma,1990,"R$ 278,00"
86,Minima Lirica (1982-1989),Paulo Henriques Britto,Claro Enigma,1989,"R$ 58,00"
